In [34]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore') 

import missingno as msno
from tqdm import tqdm_notebook

In [36]:
train_iden = pd.read_csv(r"C:\Users\EPOPCON\Desktop\Kaggle\train_identity.csv")
train_tran = pd.read_csv(r"C:\Users\EPOPCON\Desktop\Kaggle\train_transaction.csv")
test_iden = pd.read_csv(r"C:\Users\EPOPCON\Desktop\Kaggle\test_identity.csv")
test_tran = pd.read_csv(r"C:\Users\EPOPCON\Desktop\Kaggle\test_transaction.csv")

In [73]:
print (len (train_iden))
print (len (train_tran))
print (len (test_iden))
print (len (test_tran))

144233
590540
141907
506691


In [8]:
train_sample = train_tran[['isFraud','V280',
       'V284', 'V285', 'V286', 'V287', 'V290', 'V291', 'V292',
       'V293', 'V294', 'V295', 'V297', 'V298', 'V299', 'V303', 'V304', 'V305', 'V306']]
sample = train_sample.fillna(train_sample.mean())

In [40]:
train_sample = train_tran[['isFraud','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 
       'C8', 'C9', 'C10', 'C11','C12', 'C13', 'C14','D1',
                           
       'V95','V96','V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105',
       'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114',
       'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V123',
       'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132','V133', 'V134', 'V135', 'V136','V137',
                           
       'V280','V284', 'V285', 'V286', 'V287', 'V290', 'V291', 'V292',
       'V293', 'V294', 'V295', 'V297', 'V298', 'V299', 'V303', 'V304', 'V305', 'V306']]
sample = train_sample.fillna(train_sample.mean())

In [41]:
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import * 

In [68]:
import imblearn.under_sampling as usam
import imblearn.over_sampling as osam 
from sklearn.ensemble import RandomForestClassifier


X = sample.drop('isFraud', axis = 1)
Y = sample['isFraud']
feature_names = X.columns.tolist() # 컬럼을 리스트화
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y, test_size = 0.1, random_state = 13,
                                                    shuffle= False)

rus = usam.RandomUnderSampler()
undersampled_X, undersampled_Y = rus.fit_sample(Train_X, Train_Y)
model = RandomForestClassifier(n_estimators = 200, max_depth = 10, random_state = 13).fit(undersampled_X, undersampled_Y)
predicted = model.predict(Test_X)
print (roc_auc_score(Test_Y, predicted))

0.806498690153147


In [69]:
from sklearn.externals import joblib
joblib.dump(model, 'under_sampling_model.pkl')
model = joblib.load('under_sampling_model.pkl') 

In [71]:
def data_processing(data):
    train_sample = data[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 
           'C8', 'C9', 'C10', 'C11','C12', 'C13', 'C14','D1',

           'V95','V96','V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105',
           'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114',
           'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V123',
           'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132','V133', 'V134', 'V135', 'V136','V137',

           'V280','V284', 'V285', 'V286', 'V287', 'V290', 'V291', 'V292',
           'V293', 'V294', 'V295', 'V297', 'V298', 'V299', 'V303', 'V304', 'V305', 'V306']]
    sample = train_sample.fillna(train_sample.mean())
    return sample

In [75]:
test_sample = data_processing(test_tran)
len (test_sample)

506691

In [76]:
test_sample.head(1)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,V293,V294,V295,V297,V298,V299,V303,V304,V305,V306
0,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [77]:
predicted = model.predict(test_sample)

In [78]:
dic = {'TransactionID':test_tran['TransactionID'],
       'isFraud':predicted}
submission = pd.DataFrame(dic)
len (submission)

506691

In [79]:
submission.to_csv('submission.csv', index = False)